# SatMAE Finetuning on Google Colab

This notebook sets up and runs SatMAE finetuning on EuroSAT dataset using Google Colab's free GPU.

## 🚀 Features:
- Automatic environment setup
- EuroSAT dataset download and preprocessing
- SatMAE model finetuning with multispectral data
- GPU acceleration (T4/V100/A100)

**Runtime**: Make sure to select **GPU** runtime (Runtime → Change runtime type → Hardware accelerator → GPU)

## 1. Environment Setup

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Make sure to enable GPU runtime!")

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4
CUDA version: 12.4
GPU memory: 15.8 GB


In [ ]:
# Install ALL SatMAE requirements (exact versions from env.yml)
print("🔧 Installing SatMAE requirements with exact version compatibility...")
print("SatMAE requires PyTorch 1.11.0 + timm 0.3.2 (strict version requirements)")

# CRITICAL: SatMAE has strict version requirements due to API changes
# main_finetune.py contains: assert timm.__version__ == "0.3.2"
# timm 0.3.2 requires PyTorch 1.11.0 (torch._six was removed in newer versions)

print("\n⚠️ Uninstalling current PyTorch to avoid conflicts...")
!pip uninstall torch torchvision torchaudio -y

print("\n📦 Installing exact versions required by SatMAE...")
# Install PyTorch 1.11.0 with CUDA 11.3 (matching env.yml exactly)
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

# Install timm 0.3.2 (exact version required by SatMAE)
!pip install timm==0.3.2

# Core ML packages (exact versions from env.yml)
!pip install numpy==1.21.2
!pip install scipy==1.7.3
!pip install matplotlib==3.5.1
!pip install pandas==1.3.4
!pip install pillow==9.0.1

# Geospatial and data packages
!pip install rasterio==1.1.0
!pip install tqdm==4.63.0

# Logging and monitoring
!pip install tensorboard==2.6.0
!pip install wandb==0.12.14

# Additional utilities
!pip install pyyaml==6.0
!pip install requests==2.27.1

print("\n🔍 Verifying all installations...")

# Import and verify core packages
import torch
import torch.nn as nn
import torchvision
import timm
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorboard
import wandb
import yaml
import os
import requests
from pathlib import Path
from tqdm import tqdm

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ torchvision version: {torchvision.__version__}")
print(f"✅ timm version: {timm.__version__}")
print(f"✅ rasterio version: {rasterio.__version__}")
print(f"✅ numpy version: {np.__version__}")
print(f"✅ pandas version: {pd.__version__}")
print(f"✅ wandb version: {wandb.__version__}")

# Verify the critical requirement
if timm.__version__ == "0.3.2":
    print("✅ TIMM version matches SatMAE requirement (0.3.2)")
else:
    print(f"❌ TIMM version mismatch! Expected 0.3.2, got {timm.__version__}")

# Check CUDA availability
print(f"\n🔍 CUDA availability:")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA version: {torch.version.cuda}")
    print(f"✅ GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Test critical imports that were failing
try:
    from timm.models.layers import trunc_normal_
    from timm.data.mixup import Mixup
    from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
    print("✅ Critical timm imports successful")
except Exception as e:
    print(f"❌ Critical timm imports failed: {e}")

print("\n🚀 All SatMAE requirements ready for training!")

Installing required packages...
✅ timm version: 1.0.17
✅ rasterio version: 1.4.3
✅ gdown installed for Google Drive downloads
All packages installed successfully!


## 2. Download SatMAE Code and Data

In [ ]:
# Clone the SatMAE repository
!git clone https://github.com/pvinnbru/SatMAE.git
%cd SatMAE

# List repository contents
!ls -la

fatal: destination path 'SatMAE' already exists and is not an empty directory.
/content/SatMAE
total 252
drwxr-xr-x 7 root root  4096 Jul 25 13:06 .
drwxr-xr-x 1 root root  4096 Jul 25 13:06 ..
drwxr-xr-x 2 root root  4096 Jul 25 13:06 code_piet
-rw-r--r-- 1 root root  6853 Jul 25 13:06 colab_main.py
-rw-r--r-- 1 root root  1227 Jul 25 13:06 colab_setup.sh
drwxr-xr-x 2 root root  4096 Jul 25 13:06 data_splits
-rw-r--r-- 1 root root 10783 Jul 25 13:06 engine_finetune.py
-rw-r--r-- 1 root root  6019 Jul 25 13:06 engine_pretrain.py
-rw-r--r-- 1 root root  8045 Jul 25 13:06 env.yml
-rw-r--r-- 1 root root   563 Jul 25 13:06 execute.txt
-rw-r--r-- 1 root root 44308 Jul 25 13:06 file.txt
drwxr-xr-x 8 root root  4096 Jul 25 13:06 .git
-rw-r--r-- 1 root root    54 Jul 25 13:06 .gitignore
drwxr-xr-x 2 root root  4096 Jul 25 13:06 .ipynb_checkpoints
-rw-r--r-- 1 root root 19338 Jul 25 13:06 LICENSE
-rw-r--r-- 1 root root 20429 Jul 25 13:06 main_finetune.py
-rw-r--r-- 1 root root 11542 Jul 25 13:0

### 📁 Google Drive Setup

**Required Google Drive Structure:**
```
MyDrive/
├── data/                           # Unzipped EuroSAT dataset folder
│   ├── eurosat_ms/                 # Multispectral dataset
│   └── eurosat_rgb/                # RGB dataset  
└── checkpoint/
    └── pretrain-vit-large-e199.pth # Pretrained model checkpoint
```

**Setup Steps:**
1. Unzip your EuroSAT dataset and upload the `data/` folder to Google Drive root
2. Upload checkpoint to `MyDrive/checkpoint/pretrain-vit-large-e199.pth`
3. Run the cells - they will copy files to the local workspace

In [5]:
# Copy data folder from Google Drive
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Copy data folder from Google Drive to SatMAE/data
source = '/content/drive/MyDrive/data'
target = 'SatMAE/data'

if os.path.exists(target):
    shutil.rmtree(target)

shutil.copytree(source, target)
print(f"✅ Data copied from {source} to {target}")

# Verify contents
print("\n📁 Data structure:")
!ls -la SatMAE/data/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Data copied from /content/drive/MyDrive/data to SatMAE/data

📁 Data structure:
total 16
drwx------  4 root root 4096 Jul 25 13:33 .
drwxr-xr-x  8 root root 4096 Jul 25 13:47 ..
drwx------ 12 root root 4096 Jul 25 13:33 eurosat_ms
drwx------ 12 root root 4096 Jul 25 13:36 eurosat_rgb


In [6]:
# Load pretrained checkpoint from Google Drive
import os
import shutil

# Define paths
drive_checkpoint_path = '/content/drive/MyDrive/checkpoint/pretrain-vit-large-e199.pth'
local_checkpoint_dir = 'SatMAE/checkpoints'
local_checkpoint_path = 'SatMAE/checkpoints/pretrain-vit-large-e199.pth'

print("🔧 Loading pretrained checkpoint from Google Drive...")
print(f"Source: {drive_checkpoint_path}")
print(f"Target: {local_checkpoint_path}")

# Create checkpoints directory
os.makedirs(local_checkpoint_dir, exist_ok=True)
print(f"📂 Created directory: {local_checkpoint_dir}/")

# Check if checkpoint exists in Google Drive
if os.path.exists(drive_checkpoint_path):
    print(f"✅ Found checkpoint in Google Drive")
    print(f"📊 File size: {os.path.getsize(drive_checkpoint_path) / 1e6:.1f} MB")

    # Copy checkpoint to local directory
    try:
        shutil.copy2(drive_checkpoint_path, local_checkpoint_path)
        print(f"✅ Checkpoint copied successfully!")
        print(f"📁 Available at: {local_checkpoint_path}")

        # Verify the file
        if os.path.exists(local_checkpoint_path):
            print(f"✅ Verification successful - checkpoint ready for training")
        else:
            print(f"❌ Verification failed - file not found at target location")

    except Exception as e:
        print(f"❌ Copy failed: {e}")

else:
    print(f"❌ Checkpoint not found at: {drive_checkpoint_path}")
    print("Please ensure you have uploaded the checkpoint to your Google Drive")
    print("\nTo fix this:")
    print("1. Go to your Google Drive")
    print("2. Create a folder called 'checkpoint' in the root directory")
    print("3. Upload 'pretrain-vit-large-e199.pth' to MyDrive/checkpoint/")
    print("4. Run this cell again")

    print(f"\nExpected Google Drive structure:")
    print(f"  MyDrive/")
    print(f"  ├── data.zip")
    print(f"  └── checkpoint/")
    print(f"      └── pretrain-vit-large-e199.pth")

🔧 Loading pretrained checkpoint from Google Drive...
Source: /content/drive/MyDrive/checkpoint/pretrain-vit-large-e199.pth
Target: SatMAE/checkpoints/pretrain-vit-large-e199.pth
📂 Created directory: SatMAE/checkpoints/
✅ Found checkpoint in Google Drive
📊 File size: 298.8 MB
✅ Checkpoint copied successfully!
📁 Available at: SatMAE/checkpoints/pretrain-vit-large-e199.pth
✅ Verification successful - checkpoint ready for training


## 3. Data Preprocessing

### **3.1 Generate txt Files and Training Subsets**

The text files are used for loading Eurosat Data stored in `SatMAE\data\`. They look like this:

```
<path_to_image> <label>
```
For example:
```
/path/to/image1.tif    0
/path/to/image2.tif    3
...
```

The .txt-files are generate from the script below:


In [9]:
# Create train/val splits and subsets
import os
from glob import glob
import random

def generate_split_txt(root_folder, out_txt_path, split_ratio=0.8, seed=42):
    """
    Creates train/val .txt files from a root image folder organized by class.
    Supports .tif and .jpg files.
    """
    class_names = sorted(os.listdir(root_folder))
    class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

    all_samples = []
    for cls in class_names:
        tif_paths = glob(os.path.join(root_folder, cls, "*.tif"))
        jpg_paths = glob(os.path.join(root_folder, cls, "*.jpg"))
        image_paths = tif_paths + jpg_paths
        for path in image_paths:
            all_samples.append(f"{path} {class_to_idx[cls]}")

    if not all_samples:
        print(f"⚠️  No image files found in: {root_folder}")
        return

    random.seed(seed)
    random.shuffle(all_samples)
    split_idx = int(len(all_samples) * split_ratio)
    train_samples = all_samples[:split_idx]
    val_samples = all_samples[split_idx:]

    with open(out_txt_path.replace(".txt", "_train.txt"), "w") as f:
        f.write("\n".join(train_samples))
    with open(out_txt_path.replace(".txt", "_val.txt"), "w") as f:
        f.write("\n".join(val_samples))

    print(f"✅ Created train/val splits for: {root_folder}")
    print(f"   → Train: {len(train_samples)} samples")
    print(f"   → Val:   {len(val_samples)} samples")

# Execution
generate_split_txt("SatMAE/data/eurosat_ms", "SatMAE/data_splits/eurosat_ms.txt")
generate_split_txt("SatMAE/data/eurosat_rgb", "SatMAE/data_splits/eurosat_rgb.txt")

✅ Created train/val splits for: SatMAE/data/eurosat_ms
   → Train: 21600 samples
   → Val:   5400 samples
✅ Created train/val splits for: SatMAE/data/eurosat_rgb
   → Train: 21600 samples
   → Val:   5400 samples



### 3.2 **Create Training Subsets (10%, 25%, 50%, 100%)**

The Goal is to measure how model performance improves as the training data size increases. To ensure fair and meaningful comparisons across runs, the validation set remains fixed.

The following textfiles were generated and include the complete dataset:

```
SatMAE/data_splits/eurosat_ms_train.txt
SatMAE/data_splits/eurosat_rgb_train.txt
```

To subsample:

* Randomly select a percentage of lines from that file
* Save them into new files like:

  ```
  SatMAE/data_splits/eurosat_ms_train_10.txt
  SatMAE/data_splits/eurosat_ms_train_25.txt
  SatMAE/data_splits/eurosat_ms_train_50.txt
  ```

Do this for RGB and MS too:

In [11]:
def subsample_txt_file(input_path, output_prefix, percentages=[10, 25, 50], seed=42):
    with open(input_path, 'r') as f:
        lines = f.readlines()

    random.seed(seed)
    random.shuffle(lines)

    for p in percentages:
        count = int(len(lines) * (p / 100))
        subset = lines[:count]
        out_path = f"{output_prefix}_{p}.txt"
        with open(out_path, 'w') as f_out:
            f_out.writelines(subset)
        print(f"Saved {p}% subset to {out_path} ({count} samples)")


#Execution
subsample_txt_file("SatMAE/data_splits/eurosat_ms_train.txt", "SatMAE/data_splits/eurosat_ms_train", percentages=[10, 25, 50, 75])
subsample_txt_file("SatMAE/data_splits/eurosat_rgb_train.txt", "SatMAE/data_splits/eurosat_rgb_train", percentages=[10, 25, 50, 75])

print("\n✅ Data preprocessing complete!")
print("\n📁 Generated files:")
!ls -la SatMAE/data_splits/

Saved 10% subset to SatMAE/data_splits/eurosat_ms_train_10.txt (2160 samples)
Saved 25% subset to SatMAE/data_splits/eurosat_ms_train_25.txt (5400 samples)
Saved 50% subset to SatMAE/data_splits/eurosat_ms_train_50.txt (10800 samples)
Saved 75% subset to SatMAE/data_splits/eurosat_ms_train_75.txt (16200 samples)
Saved 10% subset to SatMAE/data_splits/eurosat_rgb_train_10.txt (2160 samples)
Saved 25% subset to SatMAE/data_splits/eurosat_rgb_train_25.txt (5400 samples)
Saved 50% subset to SatMAE/data_splits/eurosat_rgb_train_50.txt (10800 samples)
Saved 75% subset to SatMAE/data_splits/eurosat_rgb_train_75.txt (16200 samples)

✅ Data preprocessing complete!

📁 Generated files:
total 6732
drwxr-xr-x 3 root root    4096 Jul 25 14:14 .
drwxr-xr-x 9 root root    4096 Jul 25 14:01 ..
-rw-r--r-- 1 root root  119137 Jul 25 14:15 eurosat_ms_train_10.txt
-rw-r--r-- 1 root root  298594 Jul 25 14:15 eurosat_ms_train_25.txt
-rw-r--r-- 1 root root  595927 Jul 25 14:15 eurosat_ms_train_50.txt
-rw-r--r

In [ ]:
# Verify package imports for training
print("🔍 Verifying packages are ready for SatMAE training...")

# All packages should already be installed from Chapter 1
# Just verify imports work correctly
try:
    import torch
    import torch.nn as nn
    import torchvision
    import timm
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import os
    import sys
    import yaml
    from tqdm import tqdm
    
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ TIMM version: {timm.__version__}")
    print(f"✅ All imports successful")
    
    # Verify the critical requirement
    if timm.__version__ == "0.3.2":
        print("✅ TIMM version matches SatMAE requirement (0.3.2)")
    else:
        print(f"❌ TIMM version mismatch! Expected 0.3.2, got {timm.__version__}")
        
except Exception as e:
    print(f"❌ Import failed: {e}")
    print("⚠️ Please run Chapter 1 package installation first!")

print("🚀 Ready for SatMAE training!")

## 4. Model Training

In [ ]:
# Verify all required files exist
import os

required_files = [
    'SatMAE/main_finetune.py',
    'SatMAE/data_splits/eurosat_ms_train_10.txt',
    'SatMAE/data_splits/eurosat_ms_val.txt',
    'SatMAE/checkpoints/pretrain-vit-large-e199.pth'
]

print("Checking required files:")
all_good = True
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - MISSING")
        all_good = False

if all_good:
    print("\n🚀 All files ready for training!")
else:
    print("\n⚠️ Some files are missing. Please check the previous steps.")

In [ ]:
# Run SatMAE finetuning
# Adjust batch_size based on available GPU memory
# Check GPU memory and adjust batch size accordingly
gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0

if gpu_memory_gb >= 15:  # A100, V100
    batch_size = 16
    accum_iter = 8
elif gpu_memory_gb >= 11:  # T4 or similar
    batch_size = 8
    accum_iter = 16
else:  # Smaller GPUs
    batch_size = 4
    accum_iter = 32

print(f"🚀 GPU Memory: {gpu_memory_gb:.1f}GB")
print(f"🚀 Using batch_size={batch_size}, accum_iter={accum_iter}")

# Change to SatMAE directory and run training command
training_cmd = f"""
cd SatMAE && python main_finetune.py \
  --model_type group_c \
  --model vit_large_patch16 \
  --dataset_type euro_sat \
  --train_path data_splits/eurosat_ms_train_10.txt \
  --test_path data_splits/eurosat_ms_val.txt \
  --finetune checkpoints/pretrain-vit-large-e199.pth \
  --input_size 96 --patch_size 8 \
  --batch_size {batch_size} --accum_iter {accum_iter} \
  --epochs 30 --blr 2e-4 \
  --weight_decay 0.05 \
  --drop_path 0.2 --reprob 0.25 --mixup 0.8 --cutmix 1.0 \
  --dropped_bands 0 9 10 \
  --num_workers 2 \
  --output_dir results/eurosat_ms_10 \
  --log_dir results/eurosat_ms_10
"""

print("\n🚀 Starting training...")
print("This will take approximately 30-60 minutes depending on GPU")
print("Command:")
print(training_cmd)

# Execute training
!{training_cmd}

## 5. Monitor Training

In [ ]:
# Load TensorBoard in Colab
%load_ext tensorboard
%tensorboard --logdir SatMAE/results/eurosat_ms_10

print("TensorBoard is running above!")
print("You can monitor training progress, loss curves, and metrics.")

In [ ]:
# Check training results
import os
import glob

results_dir = "SatMAE/results/eurosat_ms_10"
if os.path.exists(results_dir):
    print("Training results:")
    !ls -la {results_dir}

    # Look for log files
    log_files = glob.glob(f"{results_dir}/*.txt")
    if log_files:
        print(f"\nLatest log file: {log_files[-1]}")
        !tail -20 {log_files[-1]}

    # Look for checkpoints
    checkpoints = glob.glob(f"{results_dir}/*.pth")
    if checkpoints:
        print(f"\nCheckpoints created: {len(checkpoints)}")
        for cp in checkpoints[-3:]:
            print(f"  {cp}")
else:
    print("No results directory found. Training may not have started yet.")

## 6. Download Results

In [ ]:
# Package results for download
import zipfile
import os
from datetime import datetime
import glob

def create_results_archive():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f"satmae_results_{timestamp}.zip"

    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add results directory from SatMAE
        results_path = 'SatMAE/results'
        if os.path.exists(results_path):
            for root, dirs, files in os.walk(results_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '.')
                    zipf.write(file_path, arcname)

        # Add training logs from SatMAE directory
        log_files = glob.glob('SatMAE/*.log')
        for log_file in log_files:
            zipf.write(log_file)

    return archive_name

if os.path.exists('SatMAE/results'):
    archive_name = create_results_archive()
    print(f"✅ Results packaged in: {archive_name}")
    print(f"File size: {os.path.getsize(archive_name) / 1e6:.1f} MB")
    print("\nYou can download this file using Colab's file panel on the left.")
else:
    print("No results to package yet.")

## 7. Optional: Cleanup and Additional Experiments

In [ ]:
# Run experiments with different data percentages
experiments = [25, 50, 75]

for pct in experiments:
    print(f"\n=== Running experiment with {pct}% of data ===")

    # Adjust epochs based on data size
    epochs = max(10, 30 - (pct // 25) * 5)  # Fewer epochs for more data

    cmd = f"""
    cd SatMAE && python main_finetune.py \
      --model_type group_c \
      --model vit_large_patch16 \
      --dataset_type euro_sat \
      --train_path data_splits/eurosat_ms_train_{pct}.txt \
      --test_path data_splits/eurosat_ms_val.txt \
      --finetune checkpoints/pretrain-vit-large-e199.pth \
      --input_size 96 --patch_size 8 \
      --batch_size {batch_size} --accum_iter {accum_iter} \
      --epochs {epochs} --blr 2e-4 \
      --weight_decay 0.05 \
      --drop_path 0.2 --reprob 0.25 --mixup 0.8 --cutmix 1.0 \
      --dropped_bands 0 9 10 \
      --num_workers 2 \
      --output_dir results/eurosat_ms_{pct} \
      --log_dir results/eurosat_ms_{pct}
    """

    print(f"Training with {pct}% data for {epochs} epochs...")
    !{cmd}

    print(f"Completed {pct}% experiment")

print("\n🎉 All experiments completed!")

In [ ]:
# Package results for download
import zipfile
import os
from datetime import datetime
import glob

def create_results_archive():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f"satmae_results_{timestamp}.zip"

    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add results directory from SatMAE
        results_path = 'SatMAE/results'
        if os.path.exists(results_path):
            for root, dirs, files in os.walk(results_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '.')
                    zipf.write(file_path, arcname)

        # Add training logs from SatMAE directory
        log_files = glob.glob('SatMAE/*.log')
        for log_file in log_files:
            zipf.write(log_file)

    return archive_name

if os.path.exists('SatMAE/results'):
    archive_name = create_results_archive()
    print(f"✅ Results packaged in: {archive_name}")
    print(f"File size: {os.path.getsize(archive_name) / 1e6:.1f} MB")
    print("\nYou can download this file using Colab's file panel on the left.")
else:
    print("No results to package yet.")